In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import getModel
import json
from datasets import CLFDataset, getDataLoader, createCSVFromFolder, getTrainTestSplit
import logging
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score
import os
import timeit
import json
from tqdm import tqdm

2023-07-18 13:34:23.882623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 13:34:24.424476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
try:
    with open(os.path.join('latest_config.json'), 'r') as f:
        config = json.load(f)
    logging.info(f'Loaded config file:\n{config}')

except:
    logging.error('Could not load config file.')


In [21]:
'''
provide the path either to: 
 a. Folder containing images or
 b. Complete path to an image
'''

path = '../validation_dataset_1/original/'


In [31]:

# create an image list

if os.path.isdir(path):
    images_ls = os.listdir(path)
    
    images_ls = [os.path.join(path, i) for i in images_ls]
    
    
    
else:
    images_ls = [path]
    

In [32]:
# create data transformation pipeline for image or a list of images

ds = CLFDataset(df = None, image_ls=images_ls, dim = config['image_size'] ,mode='eval')

In [33]:
# load the model in the memory
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = getModel(config)
model = model.to(device)
# Define the loss function
criterion = nn.CrossEntropyLoss()
# Specify the optimizer and learning rate
optimizer = optim.SGD(model.parameters(), config['lr'], config['momentum'])

best_weight = config['best_weight']

model.load_state_dict(torch.load(os.path.join('./',best_weight)))
model.eval();


In [37]:
for i in range(ds.__len__()):
    img, im_path = ds.__getimg__(i)
    inputs = img.to(device).float()

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    
    outputs = model(inputs.unsqueeze(0))
    preds = torch.softmax(outputs,1).to('cpu').tolist()[0]
    
    image_name = os.path.basename(im_path)
    
    print(f'For {image_name} probs - {preds}')
    
    

For 54349fee-f0a0-4d7d-968b-31496904cd59.png probs - [0.1494992971420288, 0.8505006432533264]
For c8c6488b-8335-4f07-a9d9-cd87ad325ad2.png probs - [0.01166511606425047, 0.9883348941802979]
For 1be815c7-a4cf-4960-bab2-4122bc1a03c2.png probs - [0.003116852603852749, 0.9968830943107605]
For 26ebae39-c728-402f-9ca4-e1c3a8d2b393.png probs - [0.11254663020372391, 0.8874533772468567]
For df0dde2d-e58d-456e-81c9-f97867c3a4a1.png probs - [0.15813779830932617, 0.8418622016906738]
For ad3489fd-ee6a-4e9b-a988-0310eb534ebd.png probs - [0.07002115994691849, 0.9299788475036621]
For 09803628-0873-4015-990c-e4ccc9665c29.png probs - [0.0035820784978568554, 0.9964178800582886]
For 2e82f557-dc3a-43ef-a131-90ca80371ecc.png probs - [0.0015000669518485665, 0.9984999895095825]
For e6b9df8d-c95f-4547-af6f-2d74ed7e4bbf.png probs - [0.004458783194422722, 0.9955411553382874]
For 7b3ac27d-0843-4821-b9d2-b558f01b0f43.png probs - [0.03146538883447647, 0.9685345888137817]
For cdc9755a-6ec9-4aea-b8cc-d2e19995dceb.png 

For df3029a9-39c4-412e-985d-278fd1b55886.png probs - [0.05880259722471237, 0.941197395324707]
For ac25d69e-f9e6-4358-9d9d-de5d2c8e028f.png probs - [0.004977232310920954, 0.9950227737426758]
For be935967-22f9-4c28-aa47-7a357f65d865.png probs - [0.05392618104815483, 0.9460738897323608]
For b2b1fb8c-0cbf-45b9-ab88-8d720a896a17.png probs - [0.1210147812962532, 0.8789852261543274]
For 5c077e77-21c0-4a30-9460-2a60e77349e5.png probs - [0.036127958446741104, 0.9638720154762268]
For ed380b4a-46bf-4805-9c37-17c59dd02130.png probs - [0.11711788922548294, 0.8828821182250977]
For 91b13b15-0684-4699-8180-0d2a7db241d7.png probs - [0.026703637093305588, 0.9732964038848877]
For bd596cf7-deeb-4a1e-87ba-90b3eb278f6e.png probs - [0.010482298210263252, 0.9895177483558655]
For cdcf2b95-9ca0-485f-878d-153fa37c0db9.png probs - [0.1455131471157074, 0.8544868230819702]
For c311d3f5-4c5e-4e97-939f-d4a472a1ce78.png probs - [0.027793219313025475, 0.972206711769104]
For a5beef27-46a5-4a93-b0cb-6b2818fff184.png prob

For dedf3fe6-ca2e-4150-a519-7e28de98e2ca.png probs - [0.006100924219936132, 0.9938991069793701]
For a0f8d157-f356-4d74-8bf2-fce6a2825bc0.png probs - [0.03224737569689751, 0.9677525758743286]
For a6928f65-403d-49a9-933b-5546a2b0e94d.png probs - [0.044797785580158234, 0.9552022218704224]
For 963c334d-acf9-431b-af4e-6480fbde7abd.png probs - [0.3278660178184509, 0.6721339225769043]
For c6d5b1b5-fffd-414e-afe2-c1773397023c.png probs - [0.10513278841972351, 0.8948672413825989]
For e9b2531c-a270-4905-b17b-9649b5f907d5.png probs - [0.06822291016578674, 0.9317770600318909]
For 90c3888a-3e6a-4835-be27-734bb8fb89e6.png probs - [0.12982149422168732, 0.8701785206794739]
For 7708e87a-b6b3-485c-be2e-371b52645234.png probs - [0.021039539948105812, 0.978960394859314]
For 64e690d3-5745-4670-9d3d-de369fc864ba.png probs - [0.0036477355752140284, 0.9963523149490356]
For bb99f471-4258-44cd-8a07-357512580d3c.png probs - [0.0038317572325468063, 0.9961681962013245]
For c472dd95-6a5a-4720-adcd-9234767a8bb8.png 

For 87865bc1-c714-4836-be5f-5c4baa6af052.png probs - [0.00854411255568266, 0.9914558529853821]
For 562a6a65-7964-4a8d-82a0-a9ecfb33848f.png probs - [0.07191365212202072, 0.9280864000320435]
For 4b6902d4-c1db-4796-9c3f-b0b97653b98f.png probs - [0.007339639123529196, 0.9926602840423584]
For a81cd459-d9b6-490c-ad51-c231a7d2a35f.png probs - [0.02098854072391987, 0.9790114760398865]
For 5d4d9d34-8d1f-4d73-a059-e7615930d92b.png probs - [0.009415297769010067, 0.990584671497345]
For 5cdfec11-7244-4490-a3ac-23f5c55095ed.png probs - [0.004704470746219158, 0.9952955842018127]
For db143469-39b3-4ee5-b11d-a1f23d9bb751.png probs - [0.02234828844666481, 0.9776517748832703]
For 0ba78950-8e1a-4ab2-9116-fe0132613527.png probs - [0.04864887148141861, 0.9513511061668396]
For b078d55e-d6f0-444d-9ee0-d210418bfc85.png probs - [0.1446211189031601, 0.8553788661956787]
For e499ac6a-2ff5-457e-aef2-d892a82d9c60.png probs - [0.02263977937400341, 0.977360188961029]
For e02b7e3d-bbca-4d8b-a9cd-98291594b4fd.png probs

For f65f2479-b3b5-46fd-b132-7936e122f664.png probs - [0.018607422709465027, 0.9813925623893738]
For c5dcacd9-b700-471a-abc7-03c746480a91.png probs - [0.12518982589244843, 0.8748101592063904]
For cf87653c-830d-4e6b-82bc-089d147580db.png probs - [0.012761673890054226, 0.9872382879257202]
For 24c46515-7b35-48b1-9c64-ab458929c4a4.png probs - [0.014064447954297066, 0.9859355688095093]
For be19afa4-2dcf-41f2-a8b7-f87a261dd4cb.png probs - [0.00749712809920311, 0.9925028085708618]
For 183cc87e-233e-485e-a41e-3439c2af5a26.png probs - [0.10090404748916626, 0.899095892906189]
For a979d112-b6ca-496f-8f40-c9344a57fc79.png probs - [0.018528856337070465, 0.9814711213111877]
For e0073a83-3e77-46c5-bdd1-c54e3a99de67.png probs - [0.003058281959965825, 0.9969417452812195]
For 3d4dea87-739c-435d-b013-2ce5e9befc41.png probs - [0.02889864705502987, 0.9711014032363892]
For a6104343-2401-46e4-91c2-c3f08cae100c.png probs - [0.018899444490671158, 0.9811005592346191]
For c6d43487-0953-4acc-8f8b-373edda5daae.png 

For 639e69b6-3696-4bdc-aa48-ba6687b42f50.png probs - [0.011192988604307175, 0.9888070821762085]
For 14855adc-05a9-4f0f-bedf-1dc153ef61cb.png probs - [0.1624470204114914, 0.8375529646873474]
For 362b13fb-a15c-43da-8fd2-e482f952e438.png probs - [0.038157690316438675, 0.9618423581123352]
For 6619040e-c624-4c98-ba06-bc1ff3eeeb0e.png probs - [0.033123601227998734, 0.9668763875961304]
For 2a456eb6-bbec-412c-a077-73aa9d1832f5.png probs - [0.6197339296340942, 0.3802660405635834]
For 79966300-d895-407f-9557-a8fab1abbfb7.png probs - [0.0025269263423979282, 0.9974730610847473]
For 9214b99f-ccd1-483d-b4c4-0527d66178d8.png probs - [0.014488587155938148, 0.9855114221572876]
For 514c6907-bc66-4c70-8da7-f372a68357b4.png probs - [0.03182315081357956, 0.9681769013404846]
For 5ee6b238-6e26-4959-9507-8c10ca322837.png probs - [0.0489005409181118, 0.9510994553565979]
For a629dc48-1cec-4ef0-a6f8-56da832318f3.png probs - [0.0055211265571415424, 0.9944788217544556]
For 362abd5a-4db6-4bda-bff3-ca173f2cb08d.png 